In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
Base.classes.keys()

['measurement', 'station']

In [4]:
measurement = Base.classes.measurement
station = Base.classes.station
session= Session(engine)

In [5]:
app = Flask(__name__)

In [6]:
@app.route("/")
def welcome():
    print("Welcome to  'Home' page...")
    return "<h1>Welcome to the Climate App!</h1> </br>" + \
            "/api/v1.0/precipitation returns temperature observations from the past year</br>" + \
            "/api/v1.0/stations returns a json list of stations</br>" + \
            "/api/v1.0/tobs returns a json list of temperature observations from the last year</br>" + \
            "/api/v1.0/&ltstart&gt and /api/v1.0/&ltstart&gt/&ltend&gt returns a json list of min, avg, and max temp - given start or start-end range"

In [7]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    prev_year = dt.date(2017,8,23)-dt.timedelta(365)

    precipitation = session.query(measurement.date, measurement.prcp).filter(measurement.date >= prev_year).all()

    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)

In [8]:
@app.route("/api/v1.0/stations")
def stations():
    print("Server received request for 'Stations' page...")
    stations = session.query(station.station).all()
    return jsonify(stations)         

In [9]:
@app.route("/api/v1.0/tobs")
def tobs():
    print("Server received request for 'Temperature Observations (tobs)' page...")
    results1 = session.query(measurement.date, measurement.tobs).filter(measurement.date>="2016-08-23").all()
    return jsonify(results1)  

In [10]:
@app.route("/api/v1.0/<start>")
def temp(start):
    print("Server received request for dynamic temperature page with only start...")
    try: 
        start_date = dt.datetime.strptime(start,"%Y-%m-%d")
        results1 = session.query(measurement.date, measurement.tobs).filter(measurement.date>=start_date).all() 
        
        df_weather = pd.DataFrame(data=results1,columns=["date","temp"])
        min_temp = df_weather["temp"].min()
        max_temp = df_weather["temp"].max()
        mean_temp = df_weather["temp"].mean()
        
        print("min_temp, mean_temp, and max_temp are returned")
        return jsonify({'TMIN': min_temp}, {'TAVG': mean_temp}, {'TMAX': max_temp}) 
    except:
        return "<p>input date is not in correct format. Dates should be formated as %Y-%m-%d</p>"

In [11]:
@app.route("/api/v1.0/<start>/<end>")
def temp_2(start,end):
    print("Server received request for dynamic temperature page with start and end...")
    try: 
        end_date = dt.datetime.strptime(end,"%Y-%m-%d")
        results1 = session.query(measurement.date, measurement.tobs).filter(measurement.date>=end_date).all()
        
        df_weather = pd.DataFrame(data=results1,columns=["date","temp"])
        min_temp = df_weather["temp"].min()
        max_temp = df_weather["temp"].max()
        mean_temp = df_weather["temp"].mean()
        
        print("min_temp, mean_temp, and max_temp are returned")
        return jsonify({'TMIN':min_temp}, {'TAVG':mean_temp}, {'TMAX':max_temp})
    except:
        print("input dates are not in correct format. Dates should be formated as %Y-%m-%d")
        

In [ ]:
if __name__ == "__main__":
    app.run()         

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
